In [3]:
import os
import matplotlib.pyplot as plt
from ultralytics import YOLO
from ipywidgets import interact, Dropdown, Checkbox, IntSlider
from PIL import Image

annotated_folder = "images/annotated"

def ensure_annotated_folder_exists():
    if not os.path.exists(annotated_folder):
        os.makedirs(annotated_folder)

def run_inference(model_path, image_folder, use_multiclass, culture_choice, conf_threshold=0.1):
    model = YOLO(model_path, task='segment')
    image_list = [f for f in os.listdir(image_folder) if f.lower().endswith(('.bmp', '.jpg', '.jpeg', '.png'))]

    if not image_list:
        print("No images found in the folder.")
        return

    def select_image(selected_image):
        image_path = os.path.join(image_folder, selected_image)
        results = model(image_path, conf=conf_threshold)
        
        if use_multiclass:
            class_to_detect = 0
            results = [r for r in results if (r.boxes is not None and class_to_detect in r.boxes.cls.int())]
        
        annotated_image = results[0].plot(labels=False, boxes=True, masks=True)
        ensure_annotated_folder_exists()
        save_path = os.path.join(annotated_folder, f"annotated_{selected_image}")
        Image.fromarray(annotated_image).save(save_path)
        print(f"Annotated image saved as {save_path}")

        class_counts = {}
        for result in results:
            if result.boxes is not None:
                class_ids = result.boxes.cls.int()
                for class_id in class_ids:
                    class_name = model.names[class_id.item()]
                    class_counts[class_name] = class_counts.get(class_name, 0) + 1

        if class_counts:
            for class_name, count in class_counts.items():
                print(f"{class_name}: {count} instances")
        else:
            print("No objects detected.")

    interact(select_image, selected_image=Dropdown(options=image_list, description="Image"))

use_multiclass = Checkbox(value=False, description='Use Multiclass Model?')
culture_choice = IntSlider(value=1, min=1, max=3, step=1, description="Culture")

def update_model_and_run(use_multiclass, culture_choice):
    if use_multiclass:
        model_path = "weights/yolo_multiclass.onnx"
        image_folder = "images/culture_1"
    else:
        if culture_choice == 1:
            model_path = "weights/mono/yolo_culture1.onnx"
        elif culture_choice == 2:
            model_path = "weights/mono/yolo_culture2.onnx"
        else:
            model_path = "weights/mono/yolo_culture3.onnx"
        
        image_folder = f"images/culture_{culture_choice}"
    
    run_inference(model_path, image_folder, use_multiclass, culture_choice)

interact(update_model_and_run, use_multiclass=use_multiclass, culture_choice=culture_choice)

interactive(children=(Checkbox(value=False, description='Use Multiclass Model?'), IntSlider(value=1, descripti…

<function __main__.update_model_and_run(use_multiclass, culture_choice)>